# Who's responsible?

Archives New Zealand's [Archway](https://archway.archives.govt.nz/) database divides government activities up into 303 functions. Over time, different agencies have been made responsible for these functions, and it can be interesting to track how these responsibilities have shifted. 

This notebook uses [data about functions harvested from Archway](harvesting_functions_from_archway.ipynb) to create a a simple visualisation of the agencies responsible for a selected function.

In [2]:
%%capture
import ipywidgets as widgets
from IPython.display import display, HTML
import json
import altair as alt
import pandas as pd

alt.renderers.enable('notebook')

In [8]:
# Load the harvested functions data from a JSON file
with open('data/functions.json', 'r') as json_file:
        data = json.load(json_file)

In [9]:
# Create a dictionary to use in the dropdown
options = {f['term']: f for f in data}

In [12]:
def make_chart(change):
    # Clear current output
    out.clear_output(wait=True)
    # Get the currently selected term from the dropdown
    term = change['new']
    # Get the agencies responsible for the selected function
    agencies = term['agencies_responsible']
    if agencies:
        # Convert to a dataframe
        df = pd.DataFrame(agencies)
        # Set some defualts for missing dates
        df['start_date'] = df['start_date'].replace('', '1853')
        df['end_date'] = df['end_date'].replace('', '2019')
        # Create a Gannt style chart
        chart = alt.Chart(df).mark_bar(size=20).encode(
            x=alt.X('start_date:T', axis=alt.Axis(format='%Y', title='Dates agency was responsible for function'), scale=alt.Scale(nice=True)),
            x2=alt.X('end_date:T'),
            y=alt.Y('entity', scale=alt.Scale(rangeStep=30), title='Agency'),
            color=alt.Color('entity', legend=None),
            tooltip=[alt.Tooltip('entity', title='Agency'), alt.Tooltip('start_date', title='From', timeUnit='year'), alt.Tooltip('end_date', title='To', timeUnit='year')]
        ).properties(
            width=700
        )
        with out:
            display(HTML('<h3>Agencies responsible for &lsquo;{}&rsquo;</h3>'.format(term['term'])))
            display(chart)
    else:
        with out:
            display(HTML('<p>No agencies responsible for &lsquo;{}&rsquo;</p>'.format(term['term'])))

# This is where the chart will be displayed
out = widgets.Output()

# Create the dropdown
term = widgets.Dropdown(
    options=options,
    value=None,
    disabled=False,
)

# Making a selection from the dropdown will automatically run 'make_chart'
term.observe(make_chart, names='value')

display(widgets.HBox([widgets.Label('Select a function:'), term]))
display(out)

Output()